## Test with *libmf*

In [1]:
from stanscofi.utils import load_dataset
from stanscofi.datasets import Dataset
from stanscofi.training_testing import weakly_correlated_split, random_simple_split, cv_training
from stanscofi.validation import compute_metrics, plot_metrics, metrics_list
import stanscofi.validation

from glob import glob
from time import time
import numpy as np
import pandas as pd

import sys #
sys.path.insert(0,"../src/") #
import benchscofi

dataset_folder="../datasets/"
! mkdir -p "../datasets/"
! pip install -q stanscofi==2.0.0
! pip freeze | grep "stanscofi"

random_state = 1234
decision_threshold=0

########################
## Dataset parameters ##
########################
dataset_names = ["TRANSCRIPT"]
split_params = {"metric": "cosine", "test_size": 0.2, "split_randomly": True}

##########################
## Algorithm parameters ##
##########################
algo_params = {
    "PMF": {
        'reg': 0.01,
        'learning_rate': 0.5,
        'n_iters': 160,
        'n_factors': 15,
        'batch_size': 100,
    },
}

/home/kali/miniconda3/envs/benchscofi/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-11 09:17:11.334820: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


DEPRECATION: pytorch-lightning 1.5.10 has a non-standard dependency specifier torch>=1.7.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
stanscofi==2.0.0


In [2]:
from subprocess import Popen
! if [ ! -d "libmf/" ]; then git clone -q https://github.com/jilljenn/libmf.git; fi
! if [ ! -s "libmf/mf-train" ] || [ ! -s "libmf/mf-predict" ]; then (cd "libmf/" && make); fi

### 4.1 Create inputs

Create the dataset, split into training/testing and validation datasets:

In [3]:
#############
## Dataset ##
#############
dataset_name = "TRANSCRIPT"

dataset_di = load_dataset(dataset_name, dataset_folder)
dataset_di.setdefault("same_item_user_features", dataset_name=="TRANSCRIPT")
dataset_di.setdefault("name", dataset_name)
dataset = Dataset(**dataset_di)

############################
## Weakly correlated sets ##
if (not split_params["split_randomly"]):
    (train_folds, test_folds), _ = weakly_correlated_split(dataset, split_params["test_size"], early_stop=1, 
                                                        metric=split_params["metric"], verbose=True)
######################
## Random splitting ##
else:
    (train_folds, test_folds), _ = random_simple_split(dataset, split_params["test_size"], 
                                                           metric=split_params["metric"])
    
from stanscofi.training_testing import random_cv_split
from sklearn.model_selection import StratifiedKFold
cv_generator = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_state)
train_dataset = dataset.subset(train_folds)
cv_folds, _ = stanscofi.training_testing.random_cv_split(train_dataset, cv_generator, metric="euclidean")

## NDGC code from libmf
import benchscofi.LibMFWrapper
from benchscofi.LibMFWrapper import execute_ndcg

from importlib import reload
reload(benchscofi.LibMFWrapper)
from benchscofi.LibMFWrapper import execute_ndcg

### 4.2 One-class matrix factorization (OCMF)

#### 4.2.a Using a stochastic gradient method

/!\ not reproducible (no random seed can be provided)

In [4]:
model_file = "libmf/ocmf_model.txt"

params_SGD = {
    'f': 10, # loss function: row-oriented pair-wise logarithmic loss
    'l2': 0.01, # L2-regularization parameter
    'k': 32, # number of dimensions
    "t": 20, # number of iterations
    "s": 10, # number of threads
    "r": 0.5, # initial learning rate
}

###############
## Training  ##
# In-memory training with holdout validation

perf_folds = []

for i, (train_folds_random_cv, test_folds_random_cv) in enumerate(cv_folds):
    
    for ii, folds in enumerate([train_folds_random_cv, test_folds_random_cv]): 
        sb_dt = train_dataset.subset(folds)
        rats = ( sb_dt.ratings.toarray()[sb_dt.folds.toarray()>0] ).ravel()
        mat = np.column_stack((sb_dt.folds.row, sb_dt.folds.col, rats))
        mat[mat<0] = 0
        if (not ii):
            mat = mat[mat[:,-1]>0] ## only positive pairs for training, positive-negative pairs for testing
        np.savetxt(("libmf/all_one_matrix_i%d.tr.txt"
                   if (not ii) else "libmf/all_one_matrix_i%d.te.txt") % (i+1), 
                   mat, fmt='%d')
        if (ii):
            y_test = (sb_dt.folds.toarray()*sb_dt.ratings.toarray())[sb_dt.folds.row, sb_dt.folds.col].ravel()
            y_test[y_test<1] = 0
 
    cmd = "libmf/mf-train -f %d -l2 %f -k %d -t %d -s %d -c %f -p %s %s %s" % tuple(
        [params_SGD[x] for x in ["f","l2","k","t","s",'r']]
                +[
                     "libmf/all_one_matrix_i%d.te.txt" % (i+1),
                      "libmf/all_one_matrix_i%d.tr.txt" % (i+1), 
                      "libmf/ocmf_model_SGD_i%d.txt" % (i+1)
                    ])
    #print(cmd)
    process = Popen(cmd.split(" "))
    process.wait()
    
    ev=0
    cmd = "libmf/mf-predict -e %d %s %s libmf/ocmf_output_SGD_e%d_i=%d.txt" % (
        ev, "libmf/all_one_matrix_i%d.te.txt" % (i+1), "libmf/ocmf_model_SGD_i%d.txt" % (i+1), ev, i+1
    )
    #print(cmd)
    process = Popen(cmd.split(" "))
    process.wait()
    
    ## Whole dataset metrics
    scores = np.loadtxt("libmf/ocmf_output_SGD_e%d_i=%d.txt" % (ev, i+1))
    #preds = np.zeros(y_test.shape)
    #preds[y_test==1] = scores
    preds = scores
    pref_fold = stanscofi.validation.AUC(y_test, preds, 0, 0)
    
    ## Row-wise metrics
    _, _, _, rowauc, _ = execute_ndcg("libmf/all_one_matrix_i%d.tr.txt" % (i+1), 
            "libmf/all_one_matrix_i%d.te.txt" % (i+1), ["libmf/ocmf_output_SGD_e%d_i=%d.txt" % (ev, i+1)])
    
    sb_dt = train_dataset.subset(test_folds_random_cv)
    from stanscofi.validation import compute_metrics 
    from scipy.sparse import coo_array
    preds = np.zeros(sb_dt.folds.shape)
    preds[sb_dt.folds.row,sb_dt.folds.col] = [max(1e-32,s) for s in scores] #1e-32
    #preds = preds.ravel()
    #preds[y_test==1] = [max(1e-32,s) for s in scores]
    preds = coo_array(preds)#.reshape(sb_dt.folds.shape))
    #predictions = (-1)**(preds.toarray().ravel()<0)
    predictions = np.zeros(sb_dt.folds.shape)
    predictions[sb_dt.folds.row,sb_dt.folds.col] = [(-1)**(max(1e-32,s)<0) for s in scores]#-1
    #predictions = predictions.ravel()
    #predictions[y_test==1] = [(-1)**(max(1e-32,s)<0) for s in scores]
    predictions = coo_array(predictions)#coo_array(predictions.reshape(sb_dt.folds.shape))
    y_pred_all = predictions.data.ravel()
    scores_all = preds.data.ravel()
    y_true_all = sb_dt.ratings.toarray()[sb_dt.folds.row,sb_dt.folds.col].ravel() 
    y_true_all[y_true_all<0] = 0
    metrics, _ = compute_metrics(preds, predictions, sb_dt, metrics=["AUC"], k=1, beta=1, verbose=False)

    print("------- METRICS")
    e=[12,13]
    for ev in e:
        cmd = "libmf/mf-predict -e %d %s %s libmf/ocmf_output_SGD_e%d_i=%d.txt" % (
            ev, "libmf/all_one_matrix_i%d.te.txt" % (i+1), "libmf/ocmf_model_SGD_i%d.txt" % (i+1), ev, i+1
        )
        #print(cmd)
        process = Popen(cmd.split(" "))
        process.wait()
    print("(rowwise stanscofi) AUC = %.3f" % metrics.values[0,0])
    print("(rowwise code) AUC = %.3f" % rowauc)
    print("(global) AUC = %.3f" % pref_fold)
    print("---------------")
    
    process = Popen((
      "rm -f libmf/all_one_matrix_i=%d.tr.txt libmf/all_one_matrix_i=%d.te.txt libmf/ocmf_output_SGD_e%d_i=%d.txt" 
                     % (i+1,i+1, ev,i+1)).split(" "))
    process.wait()
    
    perf_folds.append(pref_fold)

perf_folds

iter   tr_bprloss   va_bprloss          obj
   0       0.4164       0.6902   1.0802e+02
   1       0.4669       0.6913   1.2139e+02
   2       0.4705       0.6924   1.2252e+02
   3       0.5060       0.6930   1.3208e+02
   4       0.4937       0.6944   1.2929e+02
   5       0.4748       0.6964   1.2553e+02
   6       0.5087       0.7001   1.3486e+02
   7       0.4851       0.7018   1.2954e+02
   8       0.4446       0.7059   1.1968e+02
   9       0.4142       0.7079   1.1244e+02
  10       0.4159       0.7104   1.1320e+02
  11       0.4265       0.7116   1.1629e+02
  12       0.4318       0.7159   1.1782e+02
  13       0.4179       0.7139   1.1453e+02
  14       0.4017       0.7180   1.1068e+02
  15       0.3704       0.7207   1.0295e+02
  16       0.3562       0.7255   9.9635e+01
  17       0.3408       0.7279   9.5788e+01
  18       0.3367       0.7243   9.5055e+01
  19       0.3576       0.7287   1.0057e+02
RMSE = 0.4485


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 351.79it/s]


libmf/ocmf_output_SGD_e0_i=1.txt
ndcg = 0.031239862342423135
ndcg@10 = 0.023086149899389646
row auc = 0.055146242843101705
row-oriented auc = 0.05222606070897635
normalized row-oriented auc = 0.032001712938747665
------- METRICS
Row-wise AUC = 0.3798
Colmn-wise AUC = 0.4631
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.052
(global) AUC = 0.681
---------------
iter   tr_bprloss   va_bprloss          obj
   0       0.3120       0.6895   8.1528e+01
   1       0.4610       0.6903   1.2008e+02
   2       0.4914       0.6917   1.2890e+02
   3       0.4818       0.6935   1.2707e+02
   4       0.4461       0.6937   1.1853e+02
   5       0.4421       0.6965   1.1802e+02
   6       0.4144       0.6980   1.1131e+02
   7       0.4024       0.6989   1.0866e+02
   8       0.4085       0.7000   1.1037e+02
   9       0.4315       0.7044   1.1694e+02
  10       0.3935       0.7055   1.0793e+02
  11       0.3897       0.7099   1.0726e+02
  12       0.4103       0.7102   1.1282e+02
  13       0.

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 346.93it/s]


libmf/ocmf_output_SGD_e0_i=2.txt
ndcg = 0.02761179398248799
ndcg@10 = 0.0184715328933166
row auc = 0.0507059461765173
row-oriented auc = 0.04986887031954233
normalized row-oriented auc = 0.02563686252309735
------- METRICS
Row-wise AUC = 0.3027
Colmn-wise AUC = 0.3625
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.050
(global) AUC = 0.613
---------------
iter   tr_bprloss   va_bprloss          obj
   0       0.3707       0.6916   9.6621e+01
   1       0.5287       0.6924   1.3759e+02
   2       0.5178       0.6940   1.3532e+02
   3       0.5099       0.6945   1.3358e+02
   4       0.5014       0.6949   1.3164e+02
   5       0.4883       0.6960   1.2867e+02
   6       0.4636       0.6973   1.2267e+02
   7       0.4701       0.6995   1.2478e+02
   8       0.4503       0.7020   1.2018e+02
   9       0.4442       0.7031   1.1888e+02
  10       0.4318       0.7064   1.1610e+02
  11       0.4436       0.7086   1.2011e+02
  12       0.4114       0.7122   1.1244e+02
  13       0.4069  

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 354.22it/s]


libmf/ocmf_output_SGD_e0_i=4.txt
ndcg = 0.034875175243578915
ndcg@10 = 0.027203578025181124
row auc = 0.0567733524843435
row-oriented auc = 0.05509684000799452
normalized row-oriented auc = 0.03803741498967171
------- METRICS
Row-wise AUC = 0.4564
Colmn-wise AUC = 0.4633
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.055
(global) AUC = 0.707
---------------
iter   tr_bprloss   va_bprloss          obj
   0       0.3841       0.6918   1.0018e+02
   1       0.4146       0.6930   1.0852e+02
   2       0.4899       0.6940   1.2833e+02
   3       0.4638       0.6958   1.2211e+02
   4       0.4335       0.6973   1.1497e+02
   5       0.4281       0.6983   1.1415e+02
   6       0.4320       0.7016   1.1549e+02
   7       0.4062       0.7022   1.0921e+02
   8       0.4110       0.7048   1.1066e+02
   9       0.3953       0.7050   1.0730e+02
  10       0.3770       0.7130   1.0295e+02
  11       0.3750       0.7149   1.0276e+02
  12       0.3833       0.7181   1.0510e+02
  13       0.397

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 348.56it/s]


libmf/ocmf_output_SGD_e0_i=5.txt
ndcg = 0.03451238005240326
ndcg@10 = 0.0259044362317588
row auc = 0.0615882914497338
row-oriented auc = 0.05686080267393136
normalized row-oriented auc = 0.035891527584163495
------- METRICS
Row-wise AUC = 0.4097
Colmn-wise AUC = 0.4763
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.057
(global) AUC = 0.693
---------------


[0.6807074105955081,
 0.6126905177675301,
 0.7047303082191781,
 0.70681350027126,
 0.692781093177811]

In [5]:
################
## Validation ##
e = [10, 12] # evaluation criterion: row-oriented mean percentile rank, row-oriented area under ROC curve

## best model
best_i = np.argmax(perf_folds)

for i in range(5):
    if (i!=best_i):
        process = Popen((
            "rm -f libmf/ocmf_model_SGD_i%d.txt" % (i+1)).split(" "))
        process.wait()

# Do prediction and show evaluation value ev
for ev in e:
    print("-"*24)
    print("row-oriented mean percentile rank" if (ev==10) else "row-oriented area under ROC curve")
    
    cmd = "libmf/mf-predict -e %d %s %s libmf/ocmf_output_SGD_e%d_i=%d.txt" % (
        ev, "libmf/all_one_matrix_i%d.te.txt" % (best_i+1), 
        "libmf/ocmf_model_SGD_i%d.txt" % (best_i+1), ev, best_i+1
    )
    #print(cmd)
    process = Popen(cmd.split(" "))
    process.wait()
    
    ## Row-wise metrics
    execute_ndcg("libmf/all_one_matrix_i%d.tr.txt" % (best_i+1), "libmf/all_one_matrix_i%d.te.txt" % (best_i+1), 
                 ["libmf/ocmf_output_SGD_e%d_i=%d.txt" % (ev, best_i+1)])
    
for i in range(5):
    if (i!=best_i):
        process = Popen((
            "rm -f libmf/all_one_matrix_i%d.tr.txt libmf/all_one_matrix_i%d.te.txt" % (i+1, i+1)).split(" "))
        process.wait()

------------------------
row-oriented mean percentile rank
Row-wise MPR = 0.4865


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 352.61it/s]


libmf/ocmf_output_SGD_e10_i=4.txt
ndcg = 0.034875175243578915
ndcg@10 = 0.027203578025181124
row auc = 0.0567733524843435
row-oriented auc = 0.05509684000799452
normalized row-oriented auc = 0.03803741498967171
------------------------
row-oriented area under ROC curve
Row-wise AUC = 0.4564


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 356.09it/s]


libmf/ocmf_output_SGD_e12_i=4.txt
ndcg = 0.034875175243578915
ndcg@10 = 0.027203578025181124
row auc = 0.0567733524843435
row-oriented auc = 0.05509684000799452
normalized row-oriented auc = 0.03803741498967171


#### 4.2.b Using a coordinate descent method

In [6]:
params_CD = {
    'f': 12, # loss function: squared error (L2-norm)
    'l2': 0.01, # L2-regularization parameter
    'k': 32, # number of dimensions
    "a": 0.001, # coefficient of negative entries' loss
    "s": 10, # number of threads
    "c": 0.0001, # value of negative entries
}

###############
## Training  ##
# In-memory training with holdout validation

perf_folds = []

for i, (train_folds_random_cv, test_folds_random_cv) in enumerate(cv_folds):
    
    for ii, folds in enumerate([train_folds_random_cv, test_folds_random_cv]): 
        sb_dt = train_dataset.subset(folds)
        rats = ( sb_dt.ratings.toarray()[sb_dt.folds.toarray()>0] ).ravel()
        mat = np.column_stack((sb_dt.folds.row, sb_dt.folds.col, rats))
        mat[mat<0] = 0
        if (not ii):
            mat = mat[mat[:,-1]>0]
        np.savetxt(("libmf/all_one_matrix_i%d.tr.txt"
                   if (not ii) else "libmf/all_one_matrix_i%d.te.txt") % (i+1), 
                   mat, fmt='%d')
        if (ii):
            y_test = (sb_dt.folds.toarray()*sb_dt.ratings.toarray())[sb_dt.folds.row, sb_dt.folds.col].ravel()
            y_test[y_test<1] = 0
 
    cmd = "libmf/mf-train -f %d -l2 %f -k %d -a %f -s %d -c %f -p %s %s %s" % tuple(
        [params_CD[x] for x in ["f","l2","k","a","s",'c']]
                +[
                     "libmf/all_one_matrix_i%d.te.txt" % (i+1),
                      "libmf/all_one_matrix_i%d.tr.txt" % (i+1), 
                      "libmf/ocmf_model_CD_i%d.txt" % (i+1)
                    ])
    #print(cmd)
    process = Popen(cmd.split(" "))
    process.wait()
    
    ev=0
    cmd = "libmf/mf-predict -e %d %s %s libmf/ocmf_output_CD_e%d_i=%d.txt" % (
        ev, "libmf/all_one_matrix_i%d.te.txt" % (i+1), "libmf/ocmf_model_CD_i%d.txt" % (i+1), ev, i+1
    )
    #print(cmd)
    process = Popen(cmd.split(" "))
    process.wait()
    
    ## Whole dataset metrics
    scores = np.loadtxt("libmf/ocmf_output_CD_e%d_i=%d.txt" % (ev, i+1))
    #preds = np.zeros(y_test.shape)
    #preds[y_test==1] = scores
    preds = scores
    pref_fold = stanscofi.validation.AUC(y_test, preds, 0, 0)
    
    ## Row-wise metrics
    _, _, _, rowauc, _ = execute_ndcg("libmf/all_one_matrix_i%d.tr.txt" % (i+1), "libmf/all_one_matrix_i%d.te.txt" % (i+1), 
                 ["libmf/ocmf_output_CD_e%d_i=%d.txt" % (ev, i+1)])
    
    sb_dt = train_dataset.subset(test_folds_random_cv)
    from stanscofi.validation import compute_metrics 
    from scipy.sparse import coo_array
    preds = np.zeros(sb_dt.folds.shape)
    preds[sb_dt.folds.row,sb_dt.folds.col] = [max(1e-32,s) for s in scores] #1e-32
    #preds = preds.ravel()
    #preds[y_test==1] = [max(1e-32,s) for s in scores]
    preds = coo_array(preds)#.reshape(sb_dt.folds.shape))
    #predictions = (-1)**(preds.toarray().ravel()<0)
    predictions = np.zeros(sb_dt.folds.shape)
    predictions[sb_dt.folds.row,sb_dt.folds.col] = [(-1)**(max(1e-32,s)<0) for s in scores]#-1
    #predictions = predictions.ravel()
    #predictions[y_test==1] = [(-1)**(max(1e-32,s)<0) for s in scores]
    predictions = coo_array(predictions)#coo_array(predictions.reshape(sb_dt.folds.shape))
    y_pred_all = predictions.data.ravel()
    scores_all = preds.data.ravel()
    y_true_all = sb_dt.ratings.toarray()[sb_dt.folds.row,sb_dt.folds.col].ravel() 
    y_true_all[y_true_all<0] = 0
    metrics, _ = compute_metrics(preds, predictions, sb_dt, metrics=["AUC"], k=1, beta=1, verbose=False)
    
    print("------- METRICS")
    e=[12,13]
    for ev in e:
        cmd = "libmf/mf-predict -e %d %s %s libmf/ocmf_output_SGD_e%d_i=%d.txt" % (
            ev, "libmf/all_one_matrix_i%d.te.txt" % (i+1), "libmf/ocmf_model_SGD_i%d.txt" % (i+1), ev, i+1
        )
        #print(cmd)
        process = Popen(cmd.split(" "))
        process.wait()
    print("(rowwise stanscofi) AUC = %.3f" % metrics.values[0,0])
    print("(rowwise code) AUC = %.3f" % rowauc)
    print("(global) AUC = %.3f" % pref_fold)
    print("---------------")
    
    process = Popen((
        "rm -f libmf/all_one_matrix_i=%d.tr.txt libmf/all_one_matrix_i=%d.te.txt libmf/ocmf_output_CD_e%d_i=%d.txt" 
                     % (i+1,i+1, ev,i+1)).split(" "))
    process.wait()
    
    perf_folds.append(pref_fold)

perf_folds

iter   tr_sqerror    tr_sqerror+    tr_sqerror-   va_sqerror    va_sqerror+    va_sqerror-          obj
   0       0.8680         0.0037         0.8644    7189.8912      7189.0561         0.8351   4.5040e+00
   1       0.4912         0.0027         0.4885    7156.5181      7155.9916         0.5265   3.8823e+00
   2       0.4110         0.0029         0.4081    7126.6363      7126.1765         0.4598   3.6053e+00
   3       0.3736         0.0031         0.3705    7112.4275      7111.9994         0.4281   3.4221e+00
   4       0.3516         0.0032         0.3484    7104.1506      7103.7414         0.4092   3.2819e+00
   5       0.3389         0.0033         0.3356    7097.9611      7097.5628         0.3983   3.1696e+00
   6       0.3317         0.0034         0.3283    7093.2368      7092.8447         0.3922   3.0775e+00
   7       0.3278         0.0034         0.3244    7089.6249      7089.2360         0.3890   3.0000e+00
   8       0.3266         0.0035         0.3231    7086.4147    

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 340.99it/s]


libmf/ocmf_output_CD_e0_i=1.txt
ndcg = 0.04940704873151884
ndcg@10 = 0.044004176207183825
row auc = 0.07981145975674696
row-oriented auc = 0.06296745854604834
normalized row-oriented auc = 0.05966076275345082
------- METRICS
cannot load model from libmf/ocmf_model_SGD_i1.txt
cannot load model from libmf/ocmf_model_SGD_i1.txt
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.063
(global) AUC = 0.807
---------------
iter   tr_sqerror    tr_sqerror+    tr_sqerror-   va_sqerror    va_sqerror+    va_sqerror-          obj
   0       0.9131         0.0038         0.9093    7227.8632      7226.9893         0.8739   4.3593e+00
   1       0.5181         0.0027         0.5153    7184.6516      7184.1033         0.5483   3.7605e+00
   2       0.4198         0.0027         0.4172    7151.7181      7151.2508         0.4673   3.4946e+00
   3       0.3737         0.0028         0.3709    7135.6149      7135.1856         0.4294   3.3192e+00
   4       0.3491         0.0029         0.3462    7125.9

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 346.68it/s]


libmf/ocmf_output_CD_e0_i=2.txt
ndcg = 0.04770666387401321
ndcg@10 = 0.04219737013038118
row auc = 0.0776977509536481
row-oriented auc = 0.06212038344818786
normalized row-oriented auc = 0.05922790500202525
------- METRICS
cannot load model from libmf/ocmf_model_SGD_i2.txt
cannot load model from libmf/ocmf_model_SGD_i2.txt
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.062
(global) AUC = 0.765
---------------
iter   tr_sqerror    tr_sqerror+    tr_sqerror-   va_sqerror    va_sqerror+    va_sqerror-          obj
   0       0.9479         0.0043         0.9436    7225.7123      7224.8048         0.9075   4.6535e+00
   1       0.5216         0.0028         0.5188    7165.2595      7164.7081         0.5514   3.9522e+00
   2       0.4320         0.0029         0.4291    7128.3326      7127.8559         0.4768   3.6747e+00
   3       0.3914         0.0031         0.3884    7110.5319      7110.0882         0.4438   3.4964e+00
   4       0.3681         0.0033         0.3648    7100.894

...

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 343.48it/s]


libmf/ocmf_output_CD_e0_i=4.txt
ndcg = 0.054967093885072885
ndcg@10 = 0.05073535663622428
row auc = 0.08070275545263884
row-oriented auc = 0.06666182533860929
normalized row-oriented auc = 0.06485404017916799
------- METRICS
Row-wise AUC = 0.4564
Colmn-wise AUC = 0.4633
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.067
(global) AUC = 0.855
---------------
iter   tr_sqerror    tr_sqerror+    tr_sqerror-   va_sqerror    va_sqerror+    va_sqerror-          obj
   0       0.9227         0.0039         0.9188    7244.5924      7243.6957         0.8967   4.6299e+00
   1       0.5239         0.0029         0.5209    7178.4656      7177.9076         0.5580   3.9383e+00
   2       0.4354         0.0031         0.4324    7146.0106      7145.5262         0.4844   3.6560e+00
   3       0.3917         0.0032         0.3885    7132.2110      7131.7626         0.4485   3.4742e+00
   4       0.3680         0.0034         0.3646    7123.7328      7123.3039         0.4290   3.3381e+00
   5     

100%|█████████████████████████████████████████████████████████████████████████████████████████████| 613/613 [00:01<00:00, 328.14it/s]


libmf/ocmf_output_CD_e0_i=5.txt
ndcg = 0.06299266806870177
ndcg@10 = 0.061069656463216014
row auc = 0.08607022556880652
row-oriented auc = 0.075684113859196
normalized row-oriented auc = 0.0742018138895247
------- METRICS
cannot load model from libmf/ocmf_model_SGD_i5.txt
cannot load model from libmf/ocmf_model_SGD_i5.txt
(rowwise stanscofi) AUC = 0.500
(rowwise code) AUC = 0.076
(global) AUC = 0.854
---------------


[0.8073542530714453,
 0.7645224764342872,
 0.811618404991184,
 0.8553188152719382,
 0.854417087684796]